In [1]:
from cerebro.backend import SparkBackend
from cerebro.keras import SparkEstimator

# datas storage for intermediate data and model artifacts.
from cerebro.storage import LocalStore, HDFSStore

# Model selection/AutoML methods.
from cerebro.tune import GridSearch, RandomSearch, TPESearch

# Utility functions for specifying the search space.
from cerebro.tune import hp_choice, hp_uniform, hp_quniform, hp_loguniform, hp_qloguniform
from cerebro.tune.base import ModelSelection, update_model_results

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pyspark.sql import SparkSession
import numpy as np


spark = SparkSession \
    .builder \
    .appName("Cerebro Example") \
    .getOrCreate()

...
work_dir = '/Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/'
backend = SparkBackend(spark_context=spark.sparkContext, num_workers=1)
store = LocalStore(prefix_path=work_dir + 'test/')

df = spark.read.format("libsvm") \
    .option("numFeatures", "784") \
    .load("/Users/zijian/Desktop/ucsd/cse234/project/mnist/mnist.scale") \

from pyspark.ml.feature import OneHotEncoderEstimator

encoder = OneHotEncoderEstimator(dropLast=False)
encoder.setInputCols(["label"])
encoder.setOutputCols(["label_OHE"])

encoder_model = encoder.fit(df)
encoded = encoder_model.transform(df)

feature_columns=['features']
label_columns=['label_OHE']

train_df, test_df = encoded.randomSplit([0.8, 0.2], 100)


21/12/01 09:23:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


CEREBRO => Time: 2021-12-01 09:23:16, Running 1 Workers


In [2]:
train_df.count()

48035

In [3]:
from keras_tuner import HyperParameters
import autokeras as ak
from cerebro.nas.hphpmodel import HyperHyperModel
from keras_tuner.engine import hyperparameters


img_shape = (28, 28, 1)
num_classes = 10

hp = HyperParameters()
hp.Choice('optimizer', values=['adam'])
hp.Choice('learning_rate', values=[0.001,0.0001])
hp.Choice('batch_size', values=[32,64])

input_node = ak.ImageInput()
output_node = ak.ConvBlock(
    kernel_size=hyperparameters.Fixed('kernel_size', value=3),
    num_blocks=hyperparameters.Fixed('num_blocks', value=1),
    num_layers=hyperparameters.Fixed('num_layers', value=2),
)(input_node)
output_node = ak.ClassificationHead()(output_node)
am = HyperHyperModel(input_node, output_node, seed=2000)

am.resource_bind(
    backend=backend, 
    store=store,
    feature_columns=feature_columns,
    label_columns=label_columns,
    evaluation_metric='accuracy', 
)

am.tuner_bind(
    tuner="randomsearch", 
    hyperparameters=hp, 
    objective="val_accuracy",
    max_trials=20,
    overwrite=True,
)

_, _, meta_data, _ = am.sys_setup(train_df)

x = np.array(train_df.select(feature_columns).head(100))
y = np.array(train_df.select(label_columns).head(100))
x = [x[:,i] for i in range(x.shape[1])]
x = [r.reshape((-1, *img_shape)) for r in x]
y = np.squeeze(y,1)

print(x[0].shape)
print(y.shape)

dataset, validation_data = am._convert_to_dataset(
    x=x, y=y, validation_data=None, batch_size=32
)

am._analyze_data(dataset)
am.tuner.hyper_pipeline = None
am.tuner.hypermodel.hyper_pipeline = None
tuner = am.tuner
tuner.hypermodel.hypermodel.set_fit_args(0.2, epochs=100)

hp = tuner.oracle.get_space()
tuner._prepare_model_IO(hp, dataset=dataset)
tuner.hypermodel.build(hp)
tuner.oracle.update_space(hp)
hp = tuner.oracle.get_space()
hp.space

CEREBRO => Time: 2021-12-01 09:23:21, Preparing Data
CEREBRO => Time: 2021-12-01 09:23:22, Num Partitions: 12
CEREBRO => Time: 2021-12-01 09:23:22, Writing DataFrames
CEREBRO => Time: 2021-12-01 09:23:22, Train Data Path: file:///Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/test/intermediate_train_data
CEREBRO => Time: 2021-12-01 09:23:22, Val Data Path: file:///Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/test/intermediate_val_data


CEREBRO => Time: 2021-12-01 09:23:31, Train Partitions: 9


CEREBRO => Time: 2021-12-01 09:23:54, Val Partitions: 2


CEREBRO => Time: 2021-12-01 09:24:17, Train Rows: 38333
CEREBRO => Time: 2021-12-01 09:24:17, Val Rows: 9702
CEREBRO => Time: 2021-12-01 09:24:17, Initializing Workers
CEREBRO => Time: 2021-12-01 09:24:17, Initializing Data Loaders


2021-12-01 09:24:39.309469: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-01 09:24:39.310870: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(100, 28, 28, 1)
(100, 10)


[Choice(name: "optimizer", values: ['adam'], ordered: False, default: adam),
 Choice(name: "learning_rate", values: [0.001, 0.0001], ordered: True, default: 0.001),
 Choice(name: "batch_size", values: [32, 64], ordered: True, default: 32),
 Fixed(name: conv_block_1/kernel_size, value: 3),
 Boolean(name: "conv_block_1/separable", default: False),
 Boolean(name: "conv_block_1/max_pooling", default: True),
 Choice(name: "conv_block_1/dropout", values: [0.0, 0.25, 0.5], ordered: True, default: 0),
 Fixed(name: conv_block_1/num_blocks, value: 1),
 Fixed(name: conv_block_1/num_layers, value: 2),
 Choice(name: "conv_block_1/filters_0_0", values: [16, 32, 64, 128, 256, 512], ordered: True, default: 32),
 Choice(name: "conv_block_1/filters_0_1", values: [16, 32, 64, 128, 256, 512], ordered: True, default: 32),
 Choice(name: "classification_head_1/spatial_reduction_1/reduction_type", values: ['flatten', 'global_max', 'global_avg'], ordered: False, default: flatten),
 Choice(name: "classification

[Stage 11:>                                                         (0 + 1) / 1]

In [4]:
rel = tuner.fixed_arch_search(
    hp=hp,
    metadata=meta_data,
    epoch=5,
    x=dataset
)

0


2021-12-01 09:25:14.035005: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-01 09:25:14.394264: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-01 09:25:14.406579: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)
Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead
2021-12-01 09:25:15.366785: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
Train on 1198 steps
1198/1198 [==============================] - 41s 34ms/step - batch: 598

2021-12-01 09:28:01.352207: W tensorflow/core/framework/op_kernel.cc:1751] Invalid argument: ValueError: callback pyfunc_14 is not found
Traceback (most recent call last):

  File "/Users/zijian/.pyenv/versions/nocerebro/lib/python3.7/site-packages/tensorflow/python/ops/script_ops.py", line 238, in __call__
    raise ValueError("callback %s is not found" % token)

ValueError: callback pyfunc_14 is not found


2021-12-01 09:28:01.352590: W tensorflow/core/kernels/data/generator_dataset_op.cc:107] Error occurred when finalizing GeneratorDataset iterator: Invalid argument: ValueError: callback pyfunc_14 is not found
Traceback (most recent call last):

  File "/Users/zijian/.pyenv/versions/nocerebro/lib/python3.7/site-packages/tensorflow/python/ops/script_ops.py", line 238, in __call__
    raise ValueError("callback %s is not found" % token)

ValueError: callback pyfunc_14 is not found


	 [[{{node PyFunc}}]]
CEREBRO => Time: 2021-12-01 09:28:01, Model: model_0_1638379487, Mode: VALID, Ini

Train on 599 steps
Epoch 2/2
599/599 [==============================] - 30s 50ms/step - batch: 299.0000 - size: 1.0000 - loss: 2.1406 - accuracy: 0.1868 24s - batch: 54.5000 - - ETA: 15s - batch: 140.5000 - size: 1.0000 - loss: 2 - ETA: 12s - batch: 169.0000 - size: 1.0000 - loss - ETA: 9s - batch: 203.5000 - size: 1.0000 - loss: 2.2178 - accuracy: 0.161 - ETA: 9s - batch: 205.5000 - size: 1.0000 - loss: 2.2136 - accur - ETA: 5s - batch: 240.0000 - size: 1.0000 - loss: 2.1941 - accuracy - ETA: 3s - batch: 261.5000 - size: 1.0000 - loss: 2.1734 - accuracy: 0 - ETA: 2s - batch: 276.0000 - size: 1.0000 - loss: 2.1603 - accura
2021-12-01 09:31:25.144709: W tensorflow/core/framework/op_kernel.cc:1751] Invalid argument: ValueError: callback pyfunc_35 is not found
Traceback (most recent call last):

  File "/Users/zijian/.pyenv/versions/nocerebro/lib/python3.7/site-packages/tensorflow/python/ops/script_ops.py", line 238, in __call__
    raise ValueError("callback %s is not found" % token)

Va

Train on 599 steps
Epoch 5/5
599/599 [==============================] - 29s 49ms/step - batch: 299.0000 - size: 1.0000 - loss: 1.6998 - accuracy: 0.3341 21s - batch: 79.5000 - size: 1.0000 - loss: 1.7220 - accuracy: 0. - ETA: 20s - batch: 91.0000 - size: 1.0000 - loss: 1.7204 - accuracy - ETA: 18s - batch: 106.0000 - size: 1.00 - ETA: 13s - batch: 156.5000 - size: 1.0000 - l - ETA: 9s - batch: 197.5000 - size: 1.0000 - loss: 1.7133 - accuracy - ETA: 7s - batch: 221.0000 - size: 1.0000 - loss: 1.7103 - accurac - ETA: 5s - batch: 244.5000 - size: 1.0000 - loss: 1.7061 - accuracy: 0 - ETA: 3s - batch: 259.5000 - size: 1.0000 - loss: 1.7008 - accuracy: 0.33 - ETA: 3s - batch: 264.0000 - size: 1.0000 - loss: 1.7011 - acc
2021-12-01 09:33:33.354905: W tensorflow/core/framework/op_kernel.cc:1751] Invalid argument: ValueError: callback pyfunc_53 is not found
Traceback (most recent call last):

  File "/Users/zijian/.pyenv/versions/nocerebro/lib/python3.7/site-packages/tensorflow/python/ops/scr

...

599/599 [==============================] - 56s 93ms/step - batch: 299.0000 - size: 1.0000 - loss: 0.0793 - accuracy: 0.9763 27s - batch: 149.5000 - size: 1.0000 - loss: 0.092 - ETA: 25s - batch: 161.5000 - size: 1.0000 - loss: 0.0936 - accuracy - ETA: 24s - b - ETA: 1s - batch: 293.0000 - size: 1.0000 - loss: 0.0797 - accuracy: 0
2021-12-01 11:17:14.222425: W tensorflow/core/framework/op_kernel.cc:1751] Invalid argument: ValueError: callback pyfunc_515 is not found
Traceback (most recent call last):

  File "/Users/zijian/.pyenv/versions/nocerebro/lib/python3.7/site-packages/tensorflow/python/ops/script_ops.py", line 238, in __call__
    raise ValueError("callback %s is not found" % token)

ValueError: callback pyfunc_515 is not found


2021-12-01 11:17:14.222828: W tensorflow/core/kernels/data/generator_dataset_op.cc:107] Error occurred when finalizing GeneratorDataset iterator: Invalid argument: ValueError: callback pyfunc_515 is not found
Traceback (most recent call last):

  File "

2021-12-01 11:20:36.854033: W tensorflow/core/framework/op_kernel.cc:1751] Invalid argument: ValueError: callback pyfunc_536 is not found
Traceback (most recent call last):

  File "/Users/zijian/.pyenv/versions/nocerebro/lib/python3.7/site-packages/tensorflow/python/ops/script_ops.py", line 238, in __call__
    raise ValueError("callback %s is not found" % token)

ValueError: callback pyfunc_536 is not found


2021-12-01 11:20:36.854879: W tensorflow/core/kernels/data/generator_dataset_op.cc:107] Error occurred when finalizing GeneratorDataset iterator: Invalid argument: ValueError: callback pyfunc_536 is not found
Traceback (most recent call last):

  File "/Users/zijian/.pyenv/versions/nocerebro/lib/python3.7/site-packages/tensorflow/python/ops/script_ops.py", line 238, in __call__
    raise ValueError("callback %s is not found" % token)

ValueError: callback pyfunc_536 is not found


	 [[{{node PyFunc}}]]
CEREBRO => Time: 2021-12-01 11:20:36, Model: model_17_1638386113, Mode: VALID

2021-12-01 11:24:44.773133: W tensorflow/core/framework/op_kernel.cc:1751] Invalid argument: ValueError: callback pyfunc_554 is not found
Traceback (most recent call last):

  File "/Users/zijian/.pyenv/versions/nocerebro/lib/python3.7/site-packages/tensorflow/python/ops/script_ops.py", line 238, in __call__
    raise ValueError("callback %s is not found" % token)

ValueError: callback pyfunc_554 is not found


2021-12-01 11:24:44.773796: W tensorflow/core/kernels/data/generator_dataset_op.cc:107] Error occurred when finalizing GeneratorDataset iterator: Invalid argument: ValueError: callback pyfunc_554 is not found
Traceback (most recent call last):

  File "/Users/zijian/.pyenv/versions/nocerebro/lib/python3.7/site-packages/tensorflow/python/ops/script_ops.py", line 238, in __call__
    raise ValueError("callback %s is not found" % token)

ValueError: callback pyfunc_554 is not found


	 [[{{node PyFunc}}]]
CEREBRO => Time: 2021-12-01 11:24:44, Model: model_18_1638386439, Mode: VALID

Train on 599 steps
Epoch 2/2
599/599 [==============================] - 51s 85ms/step - batch: 299.0000 - size: 1.0000 - loss: 0.2053 - accuracy: 0.9415
2021-12-01 11:29:27.357376: W tensorflow/core/framework/op_kernel.cc:1751] Invalid argument: ValueError: callback pyfunc_575 is not found
Traceback (most recent call last):

  File "/Users/zijian/.pyenv/versions/nocerebro/lib/python3.7/site-packages/tensorflow/python/ops/script_ops.py", line 238, in __call__
    raise ValueError("callback %s is not found" % token)

ValueError: callback pyfunc_575 is not found


2021-12-01 11:29:27.358219: W tensorflow/core/kernels/data/generator_dataset_op.cc:107] Error occurred when finalizing GeneratorDataset iterator: Invalid argument: ValueError: callback pyfunc_575 is not found
Traceback (most recent call last):

  File "/Users/zijian/.pyenv/versions/nocerebro/lib/python3.7/site-packages/tensorflow/python/ops/script_ops.py", line 238, in __call__
    raise ValueError("callback %s is not found" % t

CEREBRO => Time: 2021-12-01 11:32:38, Model: model_19_1638386852, Mode: TRAIN, Initialization Time: 0.4845402240753174, Training Time: 54.658045053482056, Finalization Time: 0.18689393997192383
2021-12-01 11:32:45.129280: W tensorflow/core/framework/op_kernel.cc:1751] Invalid argument: ValueError: callback pyfunc_596 is not found
Traceback (most recent call last):

  File "/Users/zijian/.pyenv/versions/nocerebro/lib/python3.7/site-packages/tensorflow/python/ops/script_ops.py", line 238, in __call__
    raise ValueError("callback %s is not found" % token)

ValueError: callback pyfunc_596 is not found


2021-12-01 11:32:45.129677: W tensorflow/core/kernels/data/generator_dataset_op.cc:107] Error occurred when finalizing GeneratorDataset iterator: Invalid argument: ValueError: callback pyfunc_596 is not found
Traceback (most recent call last):

  File "/Users/zijian/.pyenv/versions/nocerebro/lib/python3.7/site-packages/tensorflow/python/ops/script_ops.py", line 238, in __call__
    raise 

In [5]:
# FIX archi results
rel

{'model_0_1638379487': {'trial': <keras_tuner.engine.trial.Trial at 0x169c673d0>,
  'train_loss': [2.301073292459988,
   2.0899403683331257,
   1.8755775999047721,
   1.8276395106653935,
   1.8037109942686975],
  'train_accuracy': [0.10102775692939758,
   0.20630738139152527,
   0.24277441203594208,
   0.2517216205596924,
   0.2629382312297821],
  'val_loss': [2.216241167564141,
   1.8882452963214171,
   1.8169838699855303,
   1.8146697004374706,
   1.744762895530776],
  'val_accuracy': [0.14391447603702545,
   0.2739514708518982,
   0.2751850187778473,
   0.2822779715061188,
   0.30602383613586426]},
 'model_1_1638379811': {'trial': <keras_tuner.engine.trial.Trial at 0x169c673d0>,
  'train_loss': [2.304331653703234,
   2.1406171467149,
   1.8405034746072926,
   1.7458715981155484,
   1.6997858824236365],
  'train_accuracy': [0.11800917983055115,
   0.1867956966161728,
   0.2874060869216919,
   0.3150302469730377,
   0.3341245949268341],
  'val_loss': [2.2876404382680593,
   1.86027732

[Stage 11:>                                                         (0 + 1) / 1]

In [ ]:
np 

In [ ]:
# 下面都是随便写的

In [5]:
trials = tuner.oracle.create_trials(1, tuner.tuner_id)
trial = trials[0]
est = tuner.trial_from_config_to_est(
    trial=trial,
    dataset=dataset,
    learning_rate=0.01,
    batch_size=32,
    optimizer='adam'
)
print(est.getLoss())
print(est.getOptimizer())
print(est.batch_size)

SparkEstimator_3697c2c63669__batch_size


[Stage 11:>                                                         (0 + 1) / 1]

In [5]:
for i in range(10):
    trial = tuner.oracle.create_trial(tuner_id=tuner.tuner_id)
    print(trial.trial_id)

fafdd18349fece9cd58012d506bc58c9
fafdd18349fece9cd58012d506bc58c9
fafdd18349fece9cd58012d506bc58c9
fafdd18349fece9cd58012d506bc58c9
fafdd18349fece9cd58012d506bc58c9
fafdd18349fece9cd58012d506bc58c9
fafdd18349fece9cd58012d506bc58c9
fafdd18349fece9cd58012d506bc58c9
fafdd18349fece9cd58012d506bc58c9
fafdd18349fece9cd58012d506bc58c9


In [4]:
# train using cerebro
from cerebro.tune.base import update_model_results
max_trial = 10
i = 0
ms = tuner.model_selection
while i < max_trial:
    trials = tuner.oracle.create_trials(1, tuner.tuner_id)
    trial = trials[0]
    for opt in hp._hps['optimizer'][0].values:
        for lr in hp._hps['learning_rate'][0].values:
            for bs in hp._hps['batch_size'][0].values:
                if i < max_trial:
                    estimator = tuner.trial_from_config_to_est(
                        trial=trial,
                        dataset=dataset,
                        learning_rate=lr,
                        batch_size=bs,
                        optimizer=opt
                    )
                    for epoch in range(epochs):
                    train_epoch = ms.backend.train_for_one_epoch([estimator], ms.store, None, ms.feature_cols, ms.label_cols)
                    update_model_results(est_results, train_epoch)

                    val_epoch = ms.backend.train_for_one_epoch(estimators, ms.store, dataset_idx, ms.feature_cols, ms.label_cols, is_train=False)
                    update_model_results(est_results, val_epoch)
                    self.on_epoch_end(estimators=estimators, est_resutls=est_results, epoch=epoch)
                    


7be7ced5d9d778407c4d8ff964108a20
1e126618dcc75cb311adee63175aa64c
8efe73e94bbf42daba07bb352f28e029
1464ca802ae32cc9a85f3155d756318d
0be81600d47729f26ea1b05fe7614e7e
4754450b0e3f957ea25364937c83c7f5
203ef19ab78f7b1d808fe0d22619f194
a05b22a0f2a01a45794ed0b5f78d70ed
3e4462eebce9528663dcff6a1f1bfe80
da42cd6d2c6c965b3f56d3fa13980fa3


In [6]:
tuner.oracle.hyperparameters.space

[Choice(name: "optimizer", values: ['adam'], ordered: False, default: adam),
 Choice(name: "learning_rate", values: [0.01, 0.001], ordered: True, default: 0.01),
 Choice(name: "batch_size", values: [64, 128], ordered: True, default: 64),
 Fixed(name: conv_block_1/kernel_size, value: 3),
 Boolean(name: "conv_block_1/separable", default: False),
 Boolean(name: "conv_block_1/max_pooling", default: True),
 Choice(name: "conv_block_1/dropout", values: [0.0, 0.25, 0.5], ordered: True, default: 0),
 Fixed(name: conv_block_1/num_blocks, value: 1),
 Fixed(name: conv_block_1/num_layers, value: 2),
 Choice(name: "conv_block_1/filters_0_0", values: [16, 32, 64, 128, 256, 512], ordered: True, default: 32),
 Choice(name: "conv_block_1/filters_0_1", values: [16, 32, 64, 128, 256, 512], ordered: True, default: 32),
 Choice(name: "classification_head_1/spatial_reduction_1/reduction_type", values: ['flatten', 'global_max', 'global_avg'], ordered: False, default: flatten),
 Choice(name: "classification_h

In [4]:
# using keras model to train on existing cerebro backend

In [5]:
model = keras.Sequential(
    [
        keras.Input(shape=img_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

optimizer_fake = tf.keras.optimizers.Adam(lr=0.00001)
loss = 'categorical_crossentropy'
model.compile(optimizer=optimizer_fake, loss=loss, metrics=['accuracy'])
optimizer = tf.keras.optimizers.Adam(lr=0.001)

param = {
    'model': model,
    'optimizer': optimizer, # keras opt not str
    'loss': 'categorical_crossentropy', # not sure
    'metrics': ['accuracy'],
    'batch_size': 64,
    'custom_objects': tf.keras.utils.get_custom_objects()
}

est = SparkEstimator(
        model=model,
        optimizer=param['optimizer'],
        loss=param['loss'],
        metrics=param['metrics'],
        batch_size=param['batch_size']
)
ms = tuner.model_selection
est.setFeatureCols(ms.feature_cols)
est.setLabelCols(ms.label_cols)
est.setStore(ms.store)
est.setVerbose(ms.verbose)

SparkEstimator_f8a96d6897e0

In [6]:
_, _, metadata, _ = ms.backend.prepare_data(ms.store, train_df, ms.validation, label_columns=ms.label_cols, feature_columns=ms.feature_cols)
ms.backend.initialize_workers()
ms.backend.initialize_data_loaders(ms.store, None, ms.feature_cols + ms.label_cols)

train_rel = ms.backend.train_for_one_epoch([est], ms.store, None, ms.feature_cols, ms.label_cols)

CEREBRO => Time: 2021-11-29 06:11:49, Num Partitions: 12
CEREBRO => Time: 2021-11-29 06:11:49, Writing DataFrames
CEREBRO => Time: 2021-11-29 06:11:49, Train Data Path: file:///Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/test/intermediate_train_data
CEREBRO => Time: 2021-11-29 06:11:49, Val Data Path: file:///Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/test/intermediate_val_data


CEREBRO => Time: 2021-11-29 06:11:57, Train Partitions: 9


CEREBRO => Time: 2021-11-29 06:12:14, Val Partitions: 2


CEREBRO => Time: 2021-11-29 06:12:33, Train Rows: 38634
CEREBRO => Time: 2021-11-29 06:12:33, Val Rows: 9401


[Stage 13:>                                                         (0 + 1) / 1]2021-11-29 06:12:51.851549: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-29 06:13:19.925389: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-29 06:13:19.931911: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)
Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead
2021-11-29 06:13:20.711252: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
Train on 60

In [5]:
# using AK + cerebro

In [ ]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255
x_test = x_test / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)

x_train = x_train[:30000]
y_train = y_train[:30000]

trial_rels = {}
i = 0
max_trial = 20
ts = tuner.oracle.create_trials(max_trial, tuner.tuner_id)
while i < max_trial:
    trial = ts[i]
    for lr in hp._hps['learning_rate'][0].values:
        for bs in hp._hps['batch_size'][0].values:
            if i < max_trial:
                tuner._prepare_model_IO(trial.hyperparameters, dataset=dataset)
                model = tuner.hypermodel.build(trial.hyperparameters)
                tuner.adapt(model, dataset)
                loss = 'categorical_crossentropy'
                optimizer = tf.keras.optimizers.Adam(lr=lr)
                model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])
                hist = model.fit(x_train, y_train, batch_size=bs, epochs=5, validation_split=0.2)
                trial_rels[trial.trial_id + str(i)] = hist
                print(trial.trial_id + str(i))
                i = i + 1
#         param = {
#             'model': model,
#             'optimizer': optimizer, # keras opt not str
#             'loss': loss, # not sure
#             'metrics': ['accuracy'],
#             'batch_size': bs,
#             'custom_objects': tf.keras.utils.get_custom_objects()
#         }

#         est = SparkEstimator(
#             model=model,
#             optimizer=param['optimizer'],
#             loss=param['loss'],
#             metrics=param['metrics'],
#             batch_size=param['batch_size'],
#             custom_objects=param['custom_objects']
#         )
#         ms = tuner.model_selection
#         est.setFeatureCols(ms.feature_cols)
#         est.setLabelCols(ms.label_cols)
#         est.setStore(ms.store)
#         est.setVerbose(ms.verbose)


        

(60000, 28, 28, 1)
(60000, 10)


2021-11-29 08:50:18.002566: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/5
375/375 [==============================] - 27s 69ms/step - loss: 2.2252 - accuracy: 0.1491 - val_loss: 1.8650 - val_accuracy: 0.2660
Epoch 2/5
375/375 [==============================] - 25s 67ms/step - loss: 1.6387 - accuracy: 0.3862 - val_loss: 1.0759 - val_accuracy: 0.6172
Epoch 3/5
375/375 [==============================] - 28s 74ms/step - loss: 1.0351 - accuracy: 0.6290 - val_loss: 0.8704 - val_accuracy: 0.6962
Epoch 4/5
375/375 [==============================] - 28s 74ms/step - loss: 0.8796 - accuracy: 0.6851 - val_loss: 0.7582 - val_accuracy: 0.7445
Epoch 5/5
375/375 [==============================] - 28s 74ms/step - loss: 0.7789 - accuracy: 0.7332 - val_loss: 0.5533 - val_accuracy: 0.8332
49b23e4059e1abb0c8fab15e2db406ad0
Epoch 1/5
188/188 [==============================] - 19s 100ms/step - loss: 2.3027 - accuracy: 0.1046 - val_loss: 2.3016 - val_accuracy: 0.1152
Epoch 2/5
188/188 [==============================] - 20s 105ms/step - loss: 2.3019 - accuracy: 0.1120 - val

In [8]:
ts = tuner.oracle.create_trials(4, tuner.tuner_id)

a70ab41060107e4b3ec8b2f59726a0e6


In [7]:
loss = 'categorical_crossentropy'
optimizer = tf.keras.optimizers.Adam(lr=0.001)

param = {
    'model': model,
    'optimizer': optimizer, # keras opt not str
    'loss': loss, # not sure
    'metrics': ['accuracy'],
    'batch_size': 64,
    'custom_objects': tf.keras.utils.get_custom_objects()
}

est = SparkEstimator(
    model=model,
    optimizer=param['optimizer'],
    loss=param['loss'],
    metrics=param['metrics'],
    batch_size=param['batch_size'],
    custom_objects=param['custom_objects']
)
ms = tuner.model_selection
est.setFeatureCols(ms.feature_cols)
est.setLabelCols(ms.label_cols)
est.setStore(ms.store)
est.setVerbose(ms.verbose)

SparkEstimator_b39a25611dd7

In [8]:
_, _, metadata, _ = ms.backend.prepare_data(ms.store, train_df, ms.validation, label_columns=ms.label_cols, feature_columns=ms.feature_cols)
ms.backend.initialize_workers()
ms.backend.initialize_data_loaders(ms.store, None, ms.feature_cols + ms.label_cols)

train_rel = ms.backend.train_for_one_epoch([est], ms.store, None, ms.feature_cols, ms.label_cols)

CEREBRO => Time: 2021-11-29 05:54:26, Num Partitions: 12
CEREBRO => Time: 2021-11-29 05:54:26, Writing DataFrames
CEREBRO => Time: 2021-11-29 05:54:26, Train Data Path: file:///Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/test/intermediate_train_data
CEREBRO => Time: 2021-11-29 05:54:26, Val Data Path: file:///Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/test/intermediate_val_data


CEREBRO => Time: 2021-11-29 05:54:34, Train Partitions: 9


CEREBRO => Time: 2021-11-29 05:54:52, Val Partitions: 2


CEREBRO => Time: 2021-11-29 05:55:13, Train Rows: 38374
CEREBRO => Time: 2021-11-29 05:55:13, Val Rows: 9661


[Stage 13:>                                                         (0 + 1) / 1]2021-11-29 05:55:58.561610: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-29 05:55:58.758475: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-29 05:55:58.763920: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)
Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead
2021-11-29 05:55:59.476780: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
Train on 60

In [7]:
#Result 

estimator_results = {est.getRunId(): {}}
update_model_results(estimator_results, train_rel)
model = est.create_model(estimator_results[est.getRunId()], est.getRunId(), metadata)

[Stage 13:>                                                         (0 + 1) / 1]

In [8]:
model = model.keras()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1

In [9]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255
x_test = x_test / 255

In [10]:
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)

(60000, 28, 28, 1)
(60000, 10)


In [15]:
optimizer = tf.keras.optimizers.Adam(lr=0.001)

model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=64, epochs=1, validation_split=0.2)

120/750 [===>..........................] - ETA: 3:14 - loss: 2.1722 - accuracy: 0.1737

[Stage 13:>                                                         (0 + 1) / 1]

303/750 [===========>..................] - ETA: 2:23 - loss: 1.9582 - accuracy: 0.2654

[Stage 13:>                                                         (0 + 1) / 1]

473/750 [=================>............] - ETA: 1:32 - loss: 1.8156 - accuracy: 0.3260

[Stage 13:>                                                         (0 + 1) / 1]

645/750 [========================>.....] - ETA: 35s - loss: 1.6989 - accuracy: 0.3747

[Stage 13:>                                                         (0 + 1) / 1]

750/750 [==============================] - 273s 364ms/step - loss: 1.6363 - accuracy: 0.4005 - val_loss: 0.4585 - val_accuracy: 0.8883


In [16]:
model.evaluate(x_test, y_test)

 98/313 [========>.....................] - ETA: 10s - loss: 0.5420 - accuracy: 0.8578

[Stage 13:>                                                         (0 + 1) / 1]

313/313 [==============================] - 16s 50ms/step - loss: 0.4501 - accuracy: 0.8905


[0.4500621557235718, 0.890500009059906]

[Stage 13:>                                                         (0 + 1) / 1]

In [13]:
model.predict_classes(x_test)

array([7, 2, 1, ..., 4, 5, 6])